# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
# Import the clean cities weather data file created in WeatherPy project
cleancitydatapath = "2ex_cleancitydata.csv"

# Read the clean cities weather data file created in WeatherPy project
cities_df = pd.read_csv(cleancitydatapath)
cities_df['Hotel Name'] = ""
#cities_df.head()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [45]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store "Humidity" as the weight
# HINT: be sure to handle NaN values ... #cities_complete = cities_df.dropna()
weights = cities_df["Humidity"].astype(float)

# Create Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [77]:
idealcities = cities_df.copy(deep = True)
evenbettercities = idealcities.loc[(idealcities["Temp Max"] >=18) & (idealcities["Temp Max"] <=27) & (idealcities["Wind Speed"] > 8)] 

evenbettercities.to_csv("output_data/V1ex_evenbettervacationspots.csv", index=False, header=True)
evenbettercities

,City,Lat,Lng,Temp Max,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
39,pisco,-13.70,-76.22,22.00,60,0,8.70,PE,1603392911,
49,atuona,-9.80,-139.03,25.30,76,50,8.34,PF,1603392933,
103,puerto ayora,-0.74,-90.35,25.00,78,40,9.30,EC,1603392864,
154,carnarvon,-24.87,113.63,20.00,82,0,8.20,AU,1603393123,
176,tomigusuku,26.18,127.67,23.00,83,75,13.40,JP,1603393158,
180,cidreira,-30.18,-50.21,19.05,85,98,10.43,BR,1603393166,
222,nishihara,26.18,127.76,23.00,100,90,8.70,JP,1603393242,
239,san juan bautista,-14.01,-75.74,22.00,60,0,8.70,PE,1603393276,
300,hobyo,5.35,48.53,26.76,86,72,9.26,SO,1603393380,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
# Hotel column added to dataframe at start of this script
vacationset = len(evenbettercities)
print(vacationset)

9


In [79]:

# Store latitude and longitude in locations
target_coordinates = evenbettercities[["Lat", "Lng", "City", "Country"]].copy(deep = True)
target_coordinates["Hotel Name"] = ""

# set up a parameters dictionary
params = {
    "keyword": "lodging",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
counter = 0

# use iterrows to iterate through pandas dataframe
for index, row in target_coordinates.iterrows():
    
    cityname = row['City']
    
    counter = counter + 1
    
    if counter >= 20:
        print("**Done running the first batch**")
        break
    
    if counter <= 19:
        time.sleep(1)
        print(f"Processing Record {counter} of {vacationset}. City = {cityname}")
        
        lat = row["Lat"]
        lng = row["Lng"]
        params["location"]= f'{lat}, {lng}'
            
        try:
            response = requests.get(base_url, params=params).json() 
            #print(json.dumps(response, indent=4, sort_keys=True))
            target_coordinates.loc[index, 'Hotel Name'] = response['results'][0]['name'] 
            #print(response)
           
        except (KeyError, IndexError, ValueError):
            print(f"... skipped over  " + cityname + "  because it lacked hotel options")
            pass
        
target_coordinates.head()

Processing Record 1 of 9. City = pisco
Processing Record 2 of 9. City = atuona
Processing Record 3 of 9. City = puerto ayora
Processing Record 4 of 9. City = carnarvon
Processing Record 5 of 9. City = tomigusuku
Processing Record 6 of 9. City = cidreira
Processing Record 7 of 9. City = nishihara
Processing Record 8 of 9. City = san juan bautista
Processing Record 9 of 9. City = hobyo


,Lat,Lng,City,Country,Hotel Name
39,-13.70,-76.22,pisco,PE,Apart Hotel Sol de Paracas
49,-9.80,-139.03,atuona,PF,CHALETS HANAKEE
103,-0.74,-90.35,puerto ayora,EC,Escalesia SeaFront
154,-24.87,113.63,carnarvon,AU,Carnarvon Sea Change Apartments
176,26.18,127.67,tomigusuku,JP,Hotel Collective


In [82]:
target_coordinates = target_coordinates[target_coordinates["Hotel Name"] != ""]
target_coordinates

,Lat,Lng,City,Country,Hotel Name
39,-13.70,-76.22,pisco,PE,Apart Hotel Sol de Paracas
49,-9.80,-139.03,atuona,PF,CHALETS HANAKEE
103,-0.74,-90.35,puerto ayora,EC,Escalesia SeaFront
154,-24.87,113.63,carnarvon,AU,Carnarvon Sea Change Apartments
176,26.18,127.67,tomigusuku,JP,Hotel Collective
180,-30.18,-50.21,cidreira,BR,Hotel Castelo
222,26.18,127.76,nishihara,JP,Kaiza
239,-14.01,-75.74,san juan bautista,PE,Hotel Villa de Valverde
300,5.35,48.53,hobyo,SO,Osmani Hotel


In [83]:
hotel_df = target_coordinates.copy()
hotel_df


,Lat,Lng,City,Country,Hotel Name
39,-13.70,-76.22,pisco,PE,Apart Hotel Sol de Paracas
49,-9.80,-139.03,atuona,PF,CHALETS HANAKEE
103,-0.74,-90.35,puerto ayora,EC,Escalesia SeaFront
154,-24.87,113.63,carnarvon,AU,Carnarvon Sea Change Apartments
176,26.18,127.67,tomigusuku,JP,Hotel Collective
180,-30.18,-50.21,cidreira,BR,Hotel Castelo
222,26.18,127.76,nishihara,JP,Kaiza
239,-14.01,-75.74,san juan bautista,PE,Hotel Villa de Valverde
300,5.35,48.53,hobyo,SO,Osmani Hotel


In [84]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [85]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel: {hotel}" for hotel in hotel_info]
)


fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [86]:
# Create a combined map
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))